<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/256_gpu_aydao_surgery_master_Stylegan2_Custom_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!curl https://sdk.cloud.google.com | bash

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
#!gcloud init

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-aydao-surgery-master"):
    %cd "/content/drive/MyDrive/colab-sg2-aydao-surgery-master/stylegan2-surgery"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-aydao-surgery-master
    %cd colab-sg2-aydao-surgery-master
    !git clone --branch master https://github.com/aydao/stylegan2-surgery.git
#    !git clone --branch tpu https://github.com/shawwn/stylegan2.git
#    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-surgery
    !mkdir downloads
    !mkdir datasets
    !wget -q https://raw.githubusercontent.com/aydao/stylegan2-surgery/model-release/training/networks_stylegan2.py -O ./training/networks_stylegan2.py

/content/drive/MyDrive
/content/drive/MyDrive/colab-sg2-aydao-surgery-master
Cloning into 'stylegan2-surgery'...
remote: Enumerating objects: 1557, done.
remote: Total 1557 (delta 0), reused 0 (delta 0), pack-reused 1557
Receiving objects: 100% (1557/1557), 17.03 MiB | 12.22 MiB/s, done.
Resolving deltas: 100% (1073/1073), done.
/content/drive/My Drive/colab-sg2-aydao-surgery-master/stylegan2-surgery


In [ ]:
%cd "/content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
#!git fetch origin
#!git checkout origin/main -- train.py

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/sq-256.zip"

!unzip {zip_path} -d /content/

Archive:  /content/drive/MyDrive/sq-256.zip
   creating: /content/sq-256/
  inflating: /content/sq-256/513005.jpg  
  inflating: /content/sq-256/1798005.jpg  
  inflating: /content/sq-256/2887003.jpg  
  inflating: /content/sq-256/184002.jpg  
  inflating: /content/sq-256/3078003.jpg  
  inflating: /content/sq-256/3399004.jpg  
  inflating: /content/sq-256/2385004.jpg  
  inflating: /content/sq-256/3912003.jpg  
  inflating: /content/sq-256/2613004.jpg  
  inflating: /content/sq-256/3770004.jpg  
  inflating: /content/sq-256/3045000.jpg  
  inflating: /content/sq-256/2216005.jpg  
  inflating: /content/sq-256/3153000.jpg  
  inflating: /content/sq-256/3616005.jpg  
  inflating: /content/sq-256/1807003.jpg  
  inflating: /content/sq-256/3915002.jpg  
  inflating: /content/sq-256/691002.jpg  
  inflating: /content/sq-256/307004.jpg  
  inflating: /content/sq-256/894005.jpg  
  inflating: /content/sq-256/4129003.jpg  
  inflating: /content/sq-256/2849003.jpg  
  inflating: /content/sq-256

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
#update this to the path to your image folder
dataset_path = "/content/sq-256"
#give your dataset a name
dataset_name = "sq-256"

#you don't need to edit anything here
!python dataset_tool.py create_aydao ./datasets {dataset_path} {dataset_name} 256 256
#!python dataset_tool.py create_from_images gs://train_with_tpu/datasets/{dataset_name} {dataset_path}

Loading images from "/content/sq-256"
detected 2028 images ...
Shuffle the images... using seed 123
Creating dataset "./datasets"
Adding the images to tfrecords ...
added images 0
added images 1000
added images 2000
Added 2028 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
#!export TPU_NAME={tpu_address}
#!export RESUME_PKL="/content/drive/MyDrive/sg2-ext_aydao-anime-danbooru2019s-512-5268480.pkl"
#!export RESUME_PKL=./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl
#!export RESUME_KIMG=100.4
#!export RESUME_TIME=43
#!export RESUME_TIME=`math 10*3600 + 13*60 + 49` # 10h 13m 49s
#RESUME_TIME=0*3600 + 44*60 + 31
#!export RESUME_TIME={RESUME_TIME} # 10h 13m 49s

#    resume_pkl="./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl"
#    resume_kimg=100.4
#    resume_time=0*3600 + 44*60 + 31
#    resume_pkl="/content/drive/MyDrive/stylegan2-cat-config-a.pkl"
#    resume_kimg=0
#    resume_time=0

config="config-f" # StyleGAN 2
#config="config-a" # StyleGAN 1

data_dir="./datasets"
#data_dir="/content/drive/MyDrive/colab-sg2/stylegan2/datasets"
dataset="sq-256"
mirror="False"
metrics="none"
#!export NO_GCE_CHECK=true
total_kimg=5268480+1000
#resume="/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl"
#resume="/content/drive/MyDrive/colab-sg2-aydao-surgery-master/stylegan2-surgery/results/networks/tadne-256px-5268480.pkl"
#resume="/content/drive/MyDrive/colab-sg2-aydao-surgery-master/stylegan2-surgery/results/networks/network-copyover-256px-5268480.pkl"
#resume="/content/drive/MyDrive/colab-sg2-aydao-surgery-master/stylegan2-surgery/results/00000-pretrained-networks/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl"
#resume="./results/00040-stylegan2-sq-512-1gpu-config-a"
#resume="./results/00009-stylegan2-sq-256-1gpu-config-f"


#!python run_training.py --height 512 --width 512 --resume-pkl "{resume}" --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"
#!python run_training.py --height 512 --width 512 --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"
#!python run_training.py --cond True --height 256 --width 256 --resume-pkl "{resume}" --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"
#!python run_training.py --height 256 --width 256 --resume-pkl "{resume}" --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"
!python run_training.py --height 256 --width 256 --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"


### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.

In [ ]:
!python create_initial_network_pkl.py --config config-f --height 256 --width 256 --cond False

In [ ]:

!python copy_weights_portable.py "/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl" ./results/00005-stylegan2-sq-256-1gpu-config-f/network-snapshot-000000.pkl
#!python copy_weights_portable.py "/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl" ./results/networks/network-initial-config-f-256x256-0.pkl
#!python copy_weights.py "/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl" ./results/networks/network-initial-config-f-256x256-0.pkl
#!python copy_crop_weights.py "/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl" network-initial-config-f-256x256-0.pkl
#!python copy_crop_weights.py "/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl" network-initial-config-f-512x512-0.pkl
